In [16]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from dotenv import load_dotenv
import os

load_dotenv()


True

In [17]:
ELK_HOST = os.getenv("ELK_HOST")

es = Elasticsearch(hosts=[ELK_HOST], timeout=300, max_retries=10, retry_on_timeout=True, verify_certs=False, ssl_show_warn=False)

In [19]:
index = "loadtest-webrtc-stats-merged-2"
time_start = "2023-08-01T13:48:32"
time_end = "2023-08-01T13:55:57"


def generate_qoe_data_from_elastic(index, session, userFrom, min, max):
    # query: The elasticsearch query.
    query = {
        "query": {
            "bool": {
                "must": [
                    {"exists": {"field": "vmaf"}},
                    {
                        "range": {
                            "@timestamp": {
                                "gte": min,
                                "lte": max,
                            }
                        }
                    },
                    {"match": {"userFrom": userFrom}},
                    {"match": {"session": session}},
                ]
            }
        }
    }
    # Scan function to get all the data.
    rel = scan(
        client=es,
        query=query,
        scroll="1m",
        index=index,
        raise_on_error=True,
        preserve_order=False,
        clear_scroll=True,
    )
    # We need only '_source', which has all the fields required.
    # This elimantes the elasticsearch metdata like _id, _type, _index.
    for hit in rel:
        yield hit["_source"]

def gemerate_user_in(index, session, user, min, max):
    query = {
        "query": {
            "bool": {
                "must": [
                    {
                        "range": {
                            "@timestamp": {
                                "gte": min,
                                "lte": max,
                            }
                        }
                    },
                    {"match": {"new_participant_id": user}},
                    {"match": {"new_participant_session": session}},
                ]
            }
        }
    }
    # Scan function to get all the data.
    rel = scan(
        client=es,
        query=query,
        scroll="1m",
        index=index,
        raise_on_error=True,
        preserve_order=False,
        clear_scroll=True,
    )
    # We need only '_source', which has all the fields required.
    # This elimantes the elasticsearch metdata like _id, _type, _index.
    for hit in rel:
        yield hit["_source"]
# open text file users.log and read line by line
times = []
with open("users.log", "r") as f:
    users = f.readlines()
    # parse line, separating by |
    for user in users:
        user = user.strip().split("|")
        user_time = user[0].strip()
        user_session = user[1].strip()
        user_id = user[2].strip()
        df = pd.DataFrame(generate_qoe_data_from_elastic(index, user_session, user_id, time_start, time_end))
        if df.empty:
            display("No QoE data found for user: " + user_id + " in session: " + user_session)
            # find file in qoe_files/ with user_id and session like v-LoadTestSession1-User1-User2_cuts.json
            # if not found, continue

            # convert format to timestamp Tue Aug 01 13:48:59 CEST 2023
            user_time = pd.to_datetime(
                user_time, format="%a %b %d %H:%M:%S CEST %Y"
            ).tz_localize("Europe/Madrid")


'No QoE data found for user: User2 in session: LoadTestSession6'

'No QoE data found for user: User1 in session: LoadTestSession7'

'No QoE data found for user: User1 in session: LoadTestSession1'

'No QoE data found for user: User1 in session: LoadTestSession4'

'No QoE data found for user: User2 in session: LoadTestSession1'

...

'No QoE data found for user: User2 in session: LoadTestSession20'

'No QoE data found for user: User2 in session: LoadTestSession23'

'No QoE data found for user: User2 in session: LoadTestSession22'

'No QoE data found for user: User1 in session: LoadTestSession21'

'No QoE data found for user: User1 in session: LoadTestSession31'